# MoodSync: Emotion-Responsive Lighting System

**MoodSync** is a unique project that changes the lighting in a room based on the detected emotion from a user's facial expression. The project uses a Convolutional Neural Network (CNN) to classify emotions from facial images and adjusts the brightness of Philips Hue lights accordingly.

## Project Story

The inspiration for MoodSync came from a simple moment. I was hanging out with my sister, and when I was leaving her room, she asked me to dim the lights because she was feeling down about a recent math test score. This got me thinking—what if there was a way for lights to adjust automatically based on emotions? MoodSync is the result of this thought, combining computer vision and IoT to create an emotion-responsive environment.


![FlowChart](FlowChart.png)


## Setup and Installation

To run MoodSync, you need to install the following libraries:
- **TensorFlow**: For building and training the CNN model.
- **Flask**: For creating the web app that hosts the model.
- **Requests**: To interact with the Philips Hue API.
- **Pillow**: For image processing.

In [ ]:
!pip install tensorflow flask requests pillow

In [ ]:
# TensorFlow and Keras for building the CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Flask for the web app
from flask import Flask, render_template, request, jsonify

# Requests for interacting with the Philips Hue API
import requests

# Pillow for image processing
from PIL import Image
import numpy as np

# Additional utilities
import os
import json


## Data Preparation

For this project, we use a facial emotion dataset. The dataset is organized by emotions (e.g., Happy, Sad, Neutral), and each category contains images of faces expressing that emotion.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths for training and validation data
training_dir = 'images/images/train'
validation_dir = 'images/images/validation'

# Image augmentation for training data
training_data = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=40,
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation data
validation_data = ImageDataGenerator(rescale=1./255)

# Load images from directories
training_generator = training_data.flow_from_directory(
    training_dir, target_size=(64, 64), batch_size=64, class_mode='categorical'
)

validation_generator = validation_data.flow_from_directory(
    validation_dir, target_size=(64, 64), batch_size=64, class_mode='categorical'
)


## Model Building

MoodSync uses a Convolutional Neural Network (CNN) model to classify emotions based on facial images. The model architecture includes multiple convolutional and pooling layers to extract facial features, followed by dense layers for classification.


In [ ]:
from tensorflow.keras import layers, models

# Building the model
model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(64,64,3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2), padding='same'))

model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2), padding='same'))

model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2), padding='same'))

model.add(layers.Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2), padding='same'))

model.add(layers.Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2), padding='same'))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


## Model Training

The model is trained for 50 epochs with early stopping to prevent overfitting. This helps the model to learn from the dataset effectively and stop training when improvements stop. 


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set callbacks for early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('model_weights.h5', monitor='val_accuracy', save_best_only=True)

# Train the model
history = model.fit(
    training_generator, 
    epochs=50,
    validation_data=validation_generator, 
    callbacks=[early_stopping, checkpoint]
)


## Model Evaluation

After training, we evaluate the model's performance on the validation dataset to see how well it generalizes to unseen data.


In [ ]:
# Evaluate the model
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {validation_accuracy * 100:.2f}%")


## Flask Application

The Flask application serves as the interface for MoodSync. Users upload images, and the application predicts their emotion using the CNN model. Based on the predicted emotion, the Philips Hue lights are adjusted to match the user's mood.


In [ ]:
from flask import Flask, render_template, request, jsonify
import numpy as np
from PIL import Image
import io
import tensorflow as tf
import requests

app = Flask(__name__)

# Load the CNN model
model = tf.keras.models.load_model('best_model.keras')

# Preprocess image function
def preprocess_image(image):
    try:
        img = image.resize((64, 64))  # Resize the image to 64x64 (as expected by the model)
        img_array = np.array(img)  # Convert to numpy array
        print(f"Image converted to array: {img_array.shape}")  # Check the shape of the array
        img_array = img_array / 255.0  # Normalize the image
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        return img_array
    except Exception as e:
        print(f"Error during image preprocessing: {e}")
        raise

# Brightness levels for each emotion
EMOTION_BRIGHTNESS = {
    'Happy': 90,
    'Sad': 20,
    'Neutral': 50
    # Add more mappings if you have additional emotions
}

# Philips Hue API configuration
HUE_BRIDGE_IP = '192.168.2.159'  # Replace with your Philips Hue bridge IP
LIGHT_ID = '1'  # Your light ID in the bridge
API_KEY = '0uwbM4jTAojbncWxyrQZJxiBtDKvlx23GuSjZhQ1'  # Your Philips Hue API key

def set_brightness(brightness):
    url = f"http://{HUE_BRIDGE_IP}/api/{API_KEY}/lights/{LIGHT_ID}/state"
    payload = {"bri": int((brightness / 100) * 254), "on": True}  # Convert percentage to Hue brightness scale (0-254)
    try:
        print(f"Setting brightness to {brightness}%...")
        response = requests.put(url, json=payload)
        print(f"Sending brightness adjustment request to {url} with payload {payload}")
        print(f"Response status code: {response.status_code}")
        print(f"Response content: {response.content}")
        if response.status_code == 200:
            print(f"Brightness successfully set to {brightness}%")
            return True
        else:
            print("Failed to adjust brightness.")
            return False
    except requests.exceptions.RequestException as e:
        print(f"Error adjusting light brightness: {e}")
        return False

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload_image', methods=['POST'])
def upload_image():
    try:
        # Check if image is present in the request
        if 'image' not in request.files:
            print("No image provided")
            return jsonify({'error': 'No image provided'}), 400
        
        # Read the image from the request
        file = request.files['image']
        img = Image.open(io.BytesIO(file.read()))
        print(f"Image received and loaded with size: {img.size}")

        # Preprocess the image for the CNN model
        img_array = preprocess_image(img)
        print(f"Preprocessed image shape: {img_array.shape}")

        # Get prediction from the model
        predictions = model.predict(img_array)
        print(f"Model predictions: {predictions}")
        
        predicted_class = np.argmax(predictions, axis=1)
        print(f"Predicted class index: {predicted_class}")

        # Map the class to emotion labels
        emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
        predicted_emotion = emotion_labels[predicted_class[0]]
        print(f"Predicted emotion: {predicted_emotion}")

        # Get brightness for emotion
        brightness = EMOTION_BRIGHTNESS.get(predicted_emotion, 50)  # Default to 50% if emotion not found
        print(f"Your emotion was predicted as '{predicted_emotion}'. Adjusting Philips Hue light to {brightness}% brightness.")

        # Set the brightness through Philips Hue API
        if set_brightness(brightness):
            print(f"Philips Hue light adjusted to match '{predicted_emotion}' emotion with {brightness}% brightness.")
            return jsonify({'predicted_emotion': predicted_emotion, 'brightness': brightness})
        else:
            print("Failed to adjust Philips Hue light brightness.")
            return jsonify({'predicted_emotion': predicted_emotion, 'error': 'Failed to adjust light brightness'}), 500
    
    except Exception as e:
        print(f"Error processing image: {e}")
        return jsonify({'error': f"Error processing image: {str(e)}"}), 500


if __name__ == '__main__':
    app.run(debug=True)


## Frontend Code (HTML, CSS, JavaScript)

The frontend includes a simple interface where users can capture an image, submit it to the Flask backend, and view the detected emotion. Below is the HTML, CSS, and JavaScript code for the interface.


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MoodSync</title>
    <!-- Link to the styles.css using Flask's url_for function -->
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
</head>
<body>
    <div id="welcome-screen">
        <h1 id="welcome-title">MoodSync</h1>
        <h3 id="welcome-caption" class="fade-caption">Adjust your lights based on your current emotion</h3>
    </div>
    
    <div id="main-interface" class="hidden">
        <div class="box">
            <h1>MoodSync</h1>
            <p>Adjust your lights based on your current emotion</p>

            <!-- Capture Emotion -->
            <button id="capture-button" class="capture-btn">Capture Emotion</button>


            <p id="emotion-status">Your Current Emotion is: <span id="emotion-result">None</span></p>
            
            <!-- Video Stream and Image Display -->
            <div id="video-container" class="hidden">
                <video id="camera-stream" autoplay></video>
            </div>
            <img id="selected-image" src="" alt="Captured Image" class="hidden" />
        </div>
    </div>

    <!-- Link to the app.js using Flask's url_for function -->
    <script src="{{ url_for('static', filename='app.js') }}"></script>
</body>
</html>


In [ ]:
/* General Styles */
body {
    font-family: 'Arial', sans-serif;
    margin: 0;
    padding: 0;
    background-color: #f0f0f0;
    display: flex;
    justify-content: center;
    align-items: center;
    flex-direction: column;
    height: 100vh;
}

h1, h3 {
    margin: 0;
}

.hidden {
    display: none;
}

/* Welcome Screen */
#welcome-screen {
    text-align: center;
    font-size: 2.5rem;
}

#welcome-title {
    font-size: 4rem;
    font-weight: bold;
    color: #333;
    animation: fadeInOut 2s ease-in-out infinite; /* Independent fade animation for MoodSync title */
}

#welcome-caption {
    margin-top: 20px;
    font-size: 1.5rem;
    color: #666;
}

/* Keyframes for Letter Fade-out Animation */
@keyframes fadeOutLetter {
    0% { opacity: 1; }
    100% { opacity: 0; }
}

/* Keyframes for the Title Fade-in, Fade-out */
@keyframes fadeInOut {
    0%, 100% { opacity: 1; }
    50% { opacity: 0; }
}

/* Main Interface */
#main-interface {
    text-align: center;
    margin-top: 30px;
}

.box {
    background-color: white;
    padding: 40px;
    border-radius: 15px;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
    width: 500px;
}

h1 {
    font-size: 3rem;
    color: #333;
    margin-bottom: 20px;
}

p {
    font-size: 1.2rem;
    color: #555;
}

.capture-btn, .view-backend-btn {
    font-size: 1.2rem;
    padding: 10px 20px;
    margin: 10px;
    border: none;
    border-radius: 5px;
    cursor: pointer;
    background-color: #2196f3;
    color: white;
}

#emotion-result {
    color: red;
    font-weight: bold;
}

#selected-image, #camera-stream {
    margin-top: 20px;
    max-width: 100%;
    height: auto;
    border-radius: 15px;
    border: 3px solid #ccc;
}

#camera-stream {
    width: 80%; 
    max-width: 400px;
    display: block;
    margin: 0 auto;
}


In [ ]:
document.addEventListener('DOMContentLoaded', function () {
    const welcomeScreen = document.getElementById('welcome-screen');
    const welcomeCaption = document.getElementById('welcome-caption');
    const mainInterface = document.getElementById('main-interface');

    // Wrap each letter of the caption in span tags for animation
    welcomeCaption.innerHTML = welcomeCaption.textContent.split("").map(letter => `<span>${letter}</span>`).join("");

    // Show the main interface after the caption finishes fading out
    welcomeScreen.addEventListener('click', function () {
        const letterSpans = document.querySelectorAll('.fade-caption span');
        
        // Faster letter fade-out (reduce duration)
        letterSpans.forEach((span, index) => {
            span.style.animation = `fadeOutLetter 0.3s ease-in-out forwards`; // Faster fade-out
            span.style.animationDelay = `${index * 0.05}s`; // Shorter delay between each letter
        });

        // Wait for the last letter to finish animating before switching screens
        setTimeout(() => {
            welcomeScreen.classList.add('hidden');
            mainInterface.classList.remove('hidden');
        }, letterSpans.length * 50 + 500); // Adjust timing based on faster fade-out
    });

    const captureButton = document.getElementById('capture-button');
    const backendButton = document.getElementById('view-backend');
    const emotionResult = document.getElementById('emotion-result');
    const selectedImage = document.getElementById('selected-image');
    const videoContainer = document.getElementById('video-container');
    const videoStreamElement = document.getElementById('camera-stream');

    let isCameraOpen = false;
    let videoStream = null;

    function resetUI() {
        // Hide captured image and reset UI
        selectedImage.classList.add('hidden');
        videoContainer.classList.add('hidden');
        emotionResult.innerText = 'None';
        selectedImage.src = '';
    }

    captureButton.addEventListener('click', function () {
        if (!isCameraOpen) {
            resetUI();
    
            // Open the user's camera
            navigator.mediaDevices.getUserMedia({ video: true })
                .then(function (stream) {
                    videoStream = stream;
                    videoStreamElement.srcObject = stream;
                    videoContainer.classList.remove('hidden');
                    captureButton.innerText = 'Take a Snapshot';
                    isCameraOpen = true;
                })
                .catch(function (error) {
                    alert("Unable to access camera. Please allow camera permissions.");
                });
        } else {
            // Take a snapshot
            const canvas = document.createElement('canvas');
            canvas.width = videoStreamElement.videoWidth;
            canvas.height = videoStreamElement.videoHeight;
            const ctx = canvas.getContext('2d');
            ctx.drawImage(videoStreamElement, 0, 0, canvas.width, canvas.height);
    
            // Convert the canvas image to a blob
            canvas.toBlob(function (blob) {
                const formData = new FormData();
                formData.append('image', blob, 'snapshot.jpg'); // Append the image as 'snapshot.jpg'
    
                // Send the image to the backend for processing
                fetch('/upload_image', {
                    method: 'POST',
                    body: formData
                })
                .then(response => response.json())
                .then(data => {
                    // Update the UI with the predicted emotion
                    emotionResult.innerText = data.predicted_emotion;
                    selectedImage.src = URL.createObjectURL(blob);  // Show the snapshot image
                    selectedImage.classList.remove('hidden');
                    videoContainer.classList.add('hidden');
                })
                .catch(error => {
                    console.error('Error:', error);
                    alert('Error processing the image.');
                });
            }, 'image/jpeg');
    
            // Stop the video stream
            videoStream.getTracks().forEach(track => track.stop());
    
            captureButton.innerText = 'Capture Emotion';
            isCameraOpen = false;
        }
    });

    
});


## Conclusion

MoodSync is an innovative project that demonstrates the integration of computer vision with IoT to create an emotion-responsive environment. While this is just a proof-of-concept, it shows how technology can be used to create a more personalized and responsive atmosphere.

### Future Improvements
- Extend the range of detectable emotions.
- Integrate with additional smart home systems.
- Add options for different lighting effects.

Thank you for exploring MoodSync!
